# DATAFRAME
## (explore data with Microsoft.Data.Analysis)

### Install packages / import namespaces

In [ ]:
#r "nuget:Microsoft.ML,1.5.0"
using XPlot.Plotly;
using System.Linq;

In [ ]:
#r "nuget:Microsoft.Data.Analysis"
using Microsoft.AspNetCore.Html;
using Microsoft.Data.Analysis;

#### Load object formatters from external script file Formatters.csx

In [ ]:
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Formatters.csx"

var categories = new string[] { "FlashLight", "Infrared", "Day", "Lighter" };
Formatters.Load(categories);

### Load data into data frame

In [ ]:
const string DATASET_PATH = "./sensors_data.csv";
var dataFrame = DataFrame.LoadCsv(DATASET_PATH);
dataFrame

In [ ]:
dataFrame.Description()

In [ ]:
dataFrame.Info()

In [ ]:
var sources = dataFrame.GroupBy("Source").Count();
sources

In [ ]:
dataFrame.Rows.ToList() // be careful at the size of collection! use .Take(n) when visualizing